In [1]:
import requests
from requests import Response
from pydantic import BaseModel,Field,RootModel,field_validator
import pandas as pd

youbike_url = "https://tcgbusfs.blob.core.windows.net/dotapp/youbike/v2/youbike_immediate.json"
res:Response = requests.get(youbike_url)
if res.status_code == 200:
    print("Download Success")
else:
    print("Download Failure") 

Download Success


In [2]:
class Site(BaseModel):
    站名:str = Field(alias='sna')
    地區:str = Field(alias = 'sarea')
    時間:str = Field(alias = 'mday')
    地點:str = Field(alias = 'ar')
    總車輛數:int = Field(alias='total')
    經度: float = Field(alias='longitude')
    緯度: float = Field(alias='latitude')    
    可借:int = Field(alias='available_rent_bikes')
    可還:int = Field(alias='available_return_bikes')
    狀態:bool = Field(alias='act')
    
    @field_validator("站名")
    @classmethod
    def removefilter(cls,value:str)->str:
        return value.split("_")[-1]
    
class YouBike(RootModel):
    root: list[Site]
    def __iter__(self):
        return iter(self.root)

    def __getitem__(self, item):
        return self.root[item]
    

data:YouBike = YouBike.model_validate_json(res.text)
all_sites:list[dict]= data.model_dump()

df1 = pd.DataFrame(all_sites)
df2 = df1.groupby(by='地區')[['總車輛數','可借','可還']].sum()
df2

,總車輛數,可借,可還
地區,,,
中山區,3819,1429,2354
中正區,3195,1469,1653
信義區,3324,1313,1922
內湖區,3499,1570,1884
北投區,2464,898,1517
南港區,2317,814,1495
士林區,2923,1329,1531
大同區,1629,534,1080
大安區,4975,2118,2788
